In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import structure
import sys
import pymol
from pymol import cmd
import numpy as np
import re
import time

PATH_DATA ="c:/git/apo-holo-md-analysis/apo-holo-trj-comp/data/G01/"
#PATH_DATA ="c:/git/apo-holo-md-analysis/data/apo-holo-trj-comp/"


COLORS = {
    'apo': 'blue',
    'holo': 'green',    
    'trj': 'yellow',    
    'lig': 'white'
}
POCKET_THRESHOLD = 6


class PATH:
    APO_GRO = f'{PATH_DATA}apo_2FJY/conf_wh20.gro'
    APO_TRJ = f'{PATH_DATA}apo_2FJY/traj_350ns_w_protein.xtc'
    APO_IX = f'{PATH_DATA}apo_2FJY/index.ndx'
    HOLO_GRO = f'{PATH_DATA}holo_2P70/conf_wh20.gro'
    HOLO_TRJ = f'{PATH_DATA}holo_2P70/traj_350ns_w_protein.xtc'
    HOLO_IX = f'{PATH_DATA}holo_2P70/index.ndx'

# APO_CHAIN = 'C'
# HOLO_CHAIN = 'C'
# LIGAND = 'C3S'

In [ ]:
# open a PyMOL window
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

In [ ]:
cmd.reinitialize()
cmd.pwd()
cmd.load(PATH.APO_GRO, 'trj-apo')
cmd.load_traj(PATH.APO_TRJ, 'trj-apo')
cmd.load(PATH.HOLO_GRO, 'trj-holo')
cmd.load_traj(PATH.HOLO_TRJ, 'trj-holo')


In [ ]:
cmd.align(mobile='trj-holo', target='trj-apo', mobile_state=1, target_state=1, cycles=10)
cmd.intra_fit('trj-apo')
cmd.intra_fit('trj-holo')

#cmd.zoom('all')

In [ ]:
# Extract pocket indexes

f = open(PATH.APO_IX, 'r').read()
m = re.findall(r'r_[0-9]+_*', f)
pocket_ixs_apo = [r.split('_')[1] for r in m]

f = open(PATH.HOLO_IX, 'r').read()
m = re.findall(r'r_[0-9]+_*', f)
pocket_ixs_holo = [r.split('_')[1] for r in m]

assert set(pocket_ixs_apo) == set(pocket_ixs_holo)

In [ ]:
trj_apo = pymol.cmd.get_model('trj-apo', 1)
trj_holo = pymol.cmd.get_model('trj-holo', 1)
s_apo = structure.Structure(trj_apo, 'trj-apo')
s_trj = structure.Structure(trj_holo, 'trj-holo')


In [ ]:
#Create pocket selections
pocket_res_sel = ' or '.join([f"(resi {r})" for r in pocket_ixs_apo])
def_apo = f"trj-apo and ({pocket_res_sel})"
def_holo = f"trj-holo and ({pocket_res_sel})"
cmd.create("pocket-apo", def_apo)
cmd.create("pocket-holo", def_holo)
cmd.show_as('sticks', 'pocket-apo')
cmd.show_as('sticks', 'pocket-holo')

In [ ]:
pocket_apo = structure.Structure(cmd.get_model('pocket-apo'), def_apo)
pocket_holo = structure.Structure(cmd.get_model('pocket-holo'), def_holo)

cnt_states_apo = cmd.count_states('trj-apo')
cnt_states_holo = cmd.count_states('trj-holo')

cnt_states_apo = cnt_states_holo = 1000
pocket_trj_apo = []
for i in range(cnt_states_apo):
    pocket_trj_apo.append(structure.Structure(cmd.get_model("pocket-apo", i+1), def_apo, i+1))
pocket_trj_holo = []
for i in range(cnt_states_holo):
    pocket_trj_holo.append(structure.Structure(cmd.get_model("pocket-holo", i+1), def_holo, i+1))

In [ ]:
start = time.time()

rmsd_apo_holo = np.empty(shape=(cnt_states_apo, cnt_states_holo))

for i in range(cnt_states_apo):
    #pocket_trj_apo.append(structure.Structure(cmd.get_model("pocket-apo", i+1), def_apo, i+1))
    print(f'apo-{i}')
    for j in range(i, cnt_states_holo):
        #print(f'holo-{j}')
        #cmd.align(mobile='trj-holo', target='trj-apo', mobile_state=1, target_state=1)
        #cmd.align(mobile='pocket-holo', target='pocket-apo')
        #pocket_trj_holo.append(structure.Structure(cmd.get_model("pocket-holo", i+1), def_holo, j+1))
        aux = pocket_trj_apo[i].rmsd_fast(pocket_trj_holo[j])
        # aux = cmd.align(mobile='pocket-holo', target='pocket-apo', mobile_state=i+1, target_state=j+1, transform=0,  cutoff=20000)[0]
        rmsd_apo_holo[i,j] = aux
        rmsd_apo_holo[j,i] = aux

end = time.time()
print(end - start)

In [ ]:
# cmd.align(mobile='trj-holo', target='trj-apo', mobile_state=0, target_state=0)

In [ ]:
# cmd.align(mobile='trj-holo', target='trj-apo', mobile_state=2, target_state=1, cycles=10)

In [ ]:
# cmd.align(mobile='pocket-holo', target='pocket-apo', mobile_state=1, target_state=1, cutoff=20000)

In [ ]:
# cmd.intra_fit('pocket-apo')
# cmd.intra_fit('pocket-holo')

In [ ]:
# i=50; j=50

In [ ]:
# cmd.align(mobile='pocket-holo', target='pocket-apo', mobile_state=j, target_state=i, cutoff=20000)

In [ ]:
# structure.Structure(cmd.get_model("pocket-apo", i), def_apo, i).rmsd(structure.Structure(cmd.get_model("pocket-holo", j), def_holo, j))

In [ ]:
# pocket_trj_apo[1].rmsd(pocket_trj_holo[1])